In [1]:
import pandas as pd
#import scikitlearn in that svm ie support vector machine from that svc which is Support Vector Classification.
from sklearn.svm import SVC
#it is used to split the dataset into two parts one for training and one more for testing 
from sklearn.model_selection import train_test_split
#it is used to check the accuracy of the model 
from sklearn.metrics import confusion_matrix,roc_auc_score, roc_curve,recall_score,precision_recall_curve, average_precision_score,accuracy_score, confusion_matrix, classification_report
#it is used to plot the confusion matrix and calculate the classification report
import numpy as np
import seaborn as sns#for heatmap
import matplotlib.pyplot as plt#for plotting
from dask_ml.model_selection import GridSearchCV as DaskGridSearchCV#for grid search the model
#from sklearn.feature_selection import RFECV #for feature selection

In [26]:

#creating the dataframe for the cancer csv file
cancer_data=pd.read_csv("cancer.csv")
#displaying the top most contents of it,to see the attribtes
cancer_data.head()


In [31]:
cancer_data.set_index('ID', inplace=True)

In [27]:
cancer_data=cancer_data.rename(columns={"ClumpThickness":"Clump_Thickness","Single Epithelial Cell Size":"Single_Epithelial_Cell_Size","Bare Nuclei":"Bare_Nuclei","Normal Nucleoli":"Normal_Nucleoli","Cell Size":"Cell_Size","Cell Shape":"Cell_Shape","Marginal Adhesion":"Marginal_Adhesion","Bland Chromatin":"Bland_Chromatin"},inplace=False)

In [32]:
cancer_data.head()

,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Normal_Nucleoli,Bland_Chromatin,Mitoses,Class
ID,,,,,,,,,,
1000025,5,1,1,1,2,1,3,1,1,2
1002945,5,4,4,5,7,10,3,2,1,2
1015425,3,1,1,1,2,2,3,1,1,2
1016277,6,8,8,1,3,4,3,7,1,2
1017023,4,1,1,3,2,1,3,1,1,2


In [38]:
#dropping the unnecessary data values and converting bare nuclei to int64 from object
dropping=cancer_data[ (cancer_data['Bare_Nuclei'] =="?")].index
cancer_data.drop(dropping , inplace=True)#stores the value same to the dataframe
cancer_data['Bare_Nuclei'].astype(str).astype(int)#conversion of object to string then to integer
X=cancer_data.drop("Class", axis=1)

#Stores the output value
y=cancer_data["Class"]
#train test split will split the x and y values to 4 tuples in this case we are 
#considering train size to be 80% and test size to be 20% of the data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
parameters={
              'C': [0.1, 1,5, 10,15,20,100,500],  
              'gamma': [0.5,0.80,1, 0.1],              'kernel': ['linear']}#here rbf=radial basis function 
modelsvc=SVC()  #svm model
svm_linear = DaskGridSearchCV(modelsvc, param_grid = parameters, cv = 5, n_jobs = -1)#cv is cross validation and n_jobs is for parallel processing of the model
svm_linear.fit(X_train, y_train)#fitting the model
y_pred_linear=svm_linear.predict(X_test)#predicting the model
accuracy_linear = accuracy_score(y_test, y_pred_linear)
print("Accuracy:", accuracy_linear)


Accuracy: 0.9781021897810219


In [13]:
X

,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Normal_Nucleoli,Bland_Chromatin,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


In [39]:
# pickling the model 
import pickle 
pickle_out = open("svm_linear.pkl", "wb") 
pickle.dump(svm_linear, pickle_out) 
pickle_out.close()


In [41]:
svm_linear.predict([[5,1,1,1,1,1,1,1,1]])

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([2], dtype=int64)